Annotate the growth cones in the timelapse. Load the signal channel and background, save an rgb .npy array for visualizing later. Main purpose here is however to label growth cones. Create some empty shapes layers in napari, then label manually. After that, run the SAVE axon labels cells. One can also read in already labelled axons using the LOAD cell. This also checks if a sequence of detections is continueos (no timepoints skipped).

In [22]:
import napari
from tifffile import imread
from tifffile import imsave
%gui qt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
import os

In [3]:
# timelapse 01
inp_path = '/home/loaloa/Documents/training_data/'
signal_tl140_file = inp_path+'D00_G001_RFP_compr.deflate.tif'
signal_tl140 = imread(signal_tl140_file)
H, W = signal_tl140.shape[1:]

mask_tl140_file = inp_path+'D00_G001_Transmission_compr.deflate_mask1.npy'
mask_tl140 = np.load(mask_tl140_file).astype(bool)
print(mask_tl140.shape)

signal_tl140[:, ~mask_tl140] = 0

(2920, 5764)


In [14]:
# timelapse 13 design 04_G001
signal_tl13_D04_file = inp_path+'D04_G004_GFP_compr.deflate.tif'
signal_tl13_D04 = imread(signal_tl13_D04_file)[:80]
signal_tl13_D04 -= signal_tl13_D04.min()
print(signal_tl13_D04.shape)
H2, W2 = signal_tl13_D04.shape[1:]

signal_tl13_D04 = np.pad(signal_tl13_D04, ((0,0),((H-H2)//2,(H-H2)//2),((W-W2)//2,(W-W2)//2)), mode='constant') 
print(signal_tl13_D04.shape)


mask_tl13_D04_file = inp_path+'D04_G004_Transmission_compr.deflate_mask1.npy'
mask_tl13_D04 = np.load(mask_tl13_D04_file).astype(bool)
print(mask_tl13_D04.shape)
mask_tl13_D04 = np.pad(mask_tl13_D04, (((H-H2)//2,(H-H2)//2), ((W-W2)//2,(W-W2)//2)), mode='constant') 
print(mask_tl13_D04.shape)

signal_tl13_D04[:, ~mask_tl13_D04] = 0

(80, 1972, 3868)
(80, 2920, 5764)
(1972, 3868)
(2920, 5764)


In [15]:
# timelapse 13 design 19_G035
signal_tl13_D19_file = inp_path+'D19_G035_GFP_compr.deflate.tif'
signal_tl13_D19 = imread(signal_tl13_D19_file)
signal_tl13_D19 -= signal_tl13_D19.min()
print(signal_tl13_D19.shape)
H2, W2 = signal_tl13_D19.shape[1:]
viewer.add_image(signal_tl13_D19)

(210, 1972, 3868)


<Image layer 'signal_tl13_D19 [1]' at 0x7f3f657ec070>

In [13]:
mask = viewer.layers['Labels'].data.astype(bool)[0]

In [16]:
# only a region of the timelapse was labelled. mask here
signal_tl13_D19[:, mask] = 0

signal_tl13_D19 = np.pad(signal_tl13_D19, ((0,0),((H-H2)//2,(H-H2)//2),((W-W2)//2,(W-W2)//2)), mode='constant') 
print(signal_tl13_D19.shape)


mask_tl13_D19_file = inp_path+'D19_G035_Transmission_compr.deflate_mask1.npy'
mask_tl13_D19 = np.load(mask_tl13_D19_file).astype(bool)
print(mask_tl13_D19.shape)
mask_tl13_D19 = np.pad(mask_tl13_D19, (((H-H2)//2,(H-H2)//2), ((W-W2)//2,(W-W2)//2)), mode='constant') 
print(mask_tl13_D19.shape)

signal_tl13_D19[:, ~mask_tl13_D19] = 0


(210, 2920, 5764)
(1972, 3868)
(2920, 5764)


In [17]:
# glue all dataobjects together to one tif
print(len(signal_tl140))
print(len(signal_tl13_D04))
print(len(signal_tl13_D19))
final_img = np.concatenate([signal_tl140, signal_tl13_D04, signal_tl13_D19], axis=0)

37
80
210


In [23]:
fname = f'{inp_path}/training_timelapse.tif'
# imsave(fname, final_img, photometric='minisblack', compression='deflate', bigtiff=True)
final_img = imread(signal_tl140_file)

print(final_img.shape)
# viewer.add_image(final_img)

In [21]:
# labels
tl140_labels_fname = inp_path+'D00_G001_labels_IDs_000-099.csv'
tl140_labels = pd.read_csv(tl140_labels_fname, index_col=0, header=[0,1])
ntimepoints = tl140_labels.shape[0]
print(ntimepoints)


tl13_D04_labels_fname = inp_path+'D04_G004_labels_IDs_100-166.csv'
tl13_D04_labels = pd.read_csv(tl13_D04_labels_fname, index_col=0, header=[0,1])

tl13_D04_labels.index += ntimepoints

ntimepoints = tl13_D04_labels.shape[0]
print(ntimepoints)

tl13_D19_labels_fname = inp_path+'D19_G035_labels_IDs_167-299.csv'
tl13_D19_labels = pd.read_csv(tl13_D19_labels_fname, index_col=0, header=[0,1])

tl13_D19_labels.index += ntimepoints

ntimepoints = tl13_D19_labels.shape[0]
print(ntimepoints)

all_labels = pd.concat([tl140_labels, tl13_D04_labels, tl13_D19_labels])
print(all_labels)
check = all_labels.loc[:,(slice(None),'anchor_x')].fillna(False).astype(bool)
# plt.imshow(check)

all_labels.to_csv(inp_path+'axon_anchor_labels.csv')




37
81
210
      Axon_000                                                           \
      anchor_x anchor_y bottomright_x bottomright_y     col_b     col_g   
0.0      221.0   1600.0         252.0        1627.0  0.570632  0.126446   
1.0      144.0   1625.0         170.0        1645.0  0.570632  0.126446   
2.0       83.0   1676.0         104.0        1703.0  0.570632  0.126446   
3.0      169.0   1745.0         187.0        1764.0  0.570632  0.126446   
4.0      162.0   1840.0         181.0        1861.0  0.570632  0.126446   
...        ...      ...           ...           ...       ...       ...   
286.0      NaN      NaN           NaN           NaN       NaN       NaN   
287.0      NaN      NaN           NaN           NaN       NaN       NaN   
288.0      NaN      NaN           NaN           NaN       NaN       NaN   
289.0      NaN      NaN           NaN           NaN       NaN       NaN   
290.0      NaN      NaN           NaN           NaN       NaN       NaN   

              

In [7]:
inp_path = '/run/media/loaloa/lbb_ssd/timelapse01_40_min_processed/'
inp_path2 = '/home/loaloa/Documents/timelapse13_processed/'
# inp_path = '/home/loaloa/ETZ_drive/biohybrid-signal-p/timelapse02_processed/'

# dest_dir = '/home/loaloa/Documents/tl13_D04_04_labels'
# inp_path = '/home/loaloa/Documents/'
# outp_path = '../tl140_outputdata'

# red_file = inp_path+'G001_red_compr.deflate.tif'
# red_file2 = inp_path2+'D04_G004_GFP_compr.deflate.tif'
red_file3 = inp_path2+'D19_G035_GFP_compr.deflate.tif'

# redchannel = imread(red_file)
# redchannel2 = imread(red_file2)
redchannel3 = imread(red_file3)


# redchannel2 -= redchannel2.min()
# redchannel2 = np.pad(redchannel2, ((0,0),((W-W2)//2,(W-W2)//2),((H-H2)//2,(H-H2)//2)), mode='constant') 

# tranl_file = inp_path+'G001_grey_compr.deflate.tif'
# tranl_file2 = inp_path2+'D04_G004_Transmission_compr.deflate.tif'
tranl_file3 = inp_path2+'D19_G035_Transmission_compr.deflate.tif'

# greychannel = imread(tranl_file)[0]
greychannel3 = imread(tranl_file3)[0]

In [63]:
# final = np.concatenate([redchannel, redchannel2, redchannel3])
# print(final.shape)
# viewer.add_image(final)

# maskfile = inp_path2+'D19_G035_Transmission_compr.deflate_mask1.npy'
# mask = np.load(maskfile)
# redchannel3[:, ~mask] = 0
# redchannel3 = redchannel3[:,:,:1820]

redchannel3 = np.pad(redchannel3, ((0,0),((W-W2)//2,(W-W2)//2),((H-H2)//2,(H-H2)//2)), mode='constant') 



(210, 1972, 1820)


In [4]:
%gui qt

In [6]:
import napari
viewer = napari.Viewer()

In [64]:
viewer.add_image(redchannel3)
viewer.layers['redchannel3'].blending = 'additive'
viewer.layers['redchannel3'].colormap = 'red'
viewer.layers['redchannel3'].contrast_limits = [0,2700]

# viewer.add_image(greychannel)
# viewer.layers['greychannel'].blending='additive'
# viewer.layers['greychannel'].opacity = .1
# viewer.layers['greychannel'].gamma = .6

dest_dir = '/home/loaloa/Documents/D19_G035_labels'

KeyError: "'redchannel' is not in list"

In [85]:
# compute motion and add to viewer
blur_strength = 3
lowerlim = 100
print(f'Calculating motion (clip lower lim: {lowerlim}) + '
      f'Gaussian filtering (std: {blur_strength})...', end='')

from skimage.filters import gaussian

pos_motion_seq = [np.zeros((sizey, sizex), float)]
neg_motion_seq = [np.zeros((sizey, sizex), float)]
# first timepoint has no t-1, set to all 0. Recommend to not use t=0
print()
for t in range(1, sizet):
    motion_frame = redchannel[t].astype(float) - redchannel[t-1].astype(float)
    motion_frame = gaussian(motion_frame.astype(float), blur_strength)
        
    # clipping
    motion_frame[np.abs(motion_frame)<lowerlim] = 0

    pos_motion = np.where(motion_frame>0, motion_frame, 0).astype(np.uint16)
    neg_motion = np.where(motion_frame<0, motion_frame*-1, 0).astype(np.uint16)
    
    pos_motion_seq.append(pos_motion)
    neg_motion_seq.append(neg_motion)
    
print('Done.')

pos_motion_seq = np.stack(pos_motion_seq).astype(np.uint16)
neg_motion_seq = np.stack(neg_motion_seq).astype(np.uint16)

Calculating motion (clip lower lim: 100) + Gaussian filtering (std: 3)...


KeyboardInterrupt: 

In [ ]:
viewer.add_image(pos_motion_seq)
viewer.layers['pos_motion_seq'].blending = 'additive'
viewer.layers['pos_motion_seq'].colormap = 'green'
viewer.layers['pos_motion_seq'].contrast_limits = [0,2700]

viewer.add_image(neg_motion_seq)
viewer.layers['neg_motion_seq'].blending = 'additive'
viewer.layers['neg_motion_seq'].colormap = 'blue'
viewer.layers['neg_motion_seq'].contrast_limits = [0,2700]


In [57]:
# CREATE EMPTY AXON LABELS
axons_index = range(290,300)

for ax_i in axons_index:
    name = f'Axon_{ax_i:0>3}'
    col = np.random.rand(1,3)

    # edge coloring doesn't work unfortunately, do manually in gui
    viewer.add_points(name=name, edge_color=col, face_color=col, opacity=.25, size=35, ndim=3)

In [58]:
# save labels
axons_index = range(259,300)

all_axons_data = []
for ax_i in axons_index:
    name = f'Axon_{ax_i:0>3}'
    dat = viewer.layers[name].data
    idx = pd.MultiIndex.from_product([(name,), ('anchor_x', 'anchor_y')])
    data = pd.DataFrame(dat[:,1:], index=dat[:,0], columns=idx, dtype=int)
    
    if ax_i == 137:
        print(data.to_string())
    if data.empty:
        print(dat)
        print(name)
    # print(data)
    all_axons_data.append(data)

# print(all_axons_data)
all_axons_data = pd.concat(all_axons_data, axis=1)

fname = f'{dest_dir}/labels_ID{axons_index[0]}-{axons_index[-1]}.csv'
all_axons_data.to_csv(fname) 
print(fname)
all_axons_data

/home/loaloa/Documents/D19_G035_labels/labels_ID259-299.csv


Axon_259          Axon_260          Axon_261          Axon_262           \
      anchor_x anchor_y anchor_x anchor_y anchor_x anchor_y anchor_x anchor_y   
0.0      503.0   1747.0    964.0   1691.0      NaN      NaN      NaN      NaN   
1.0      506.0   1748.0    938.0   1693.0      NaN      NaN      NaN      NaN   
2.0      503.0   1747.0    873.0   1681.0      NaN      NaN      NaN      NaN   
3.0      505.0   1748.0    838.0   1693.0      NaN      NaN      NaN      NaN   
4.0      491.0   1746.0    819.0   1726.0      NaN      NaN      NaN      NaN   
...        ...      ...      ...      ...      ...      ...      ...      ...   
205.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
206.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
207.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
208.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
209.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

      Axon_263           ... Axon_295          Axon_296          Axon_297  \
      anchor_x anchor_y  ... anchor_x anchor_y anchor_x anchor_y anchor_x   
0.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
1.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
2.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
3.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
4.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
...        ...      ...  ...      ...      ...      ...      ...      ...   
205.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    546.0   
206.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    547.0   
207.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    545.0   
208.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    547.0   
209.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    544.0   

               Axon_298          Axon_299           
      anchor_y anchor_x anchor_y anchor_x anchor_y  
0.0        NaN      NaN      NaN      NaN      NaN  
1.0        NaN      NaN      NaN      NaN      NaN  
2.0        NaN      NaN      NaN      NaN      NaN  
3.0        NaN      NaN      NaN      NaN      NaN  
4.0        NaN      NaN      NaN      NaN      NaN  
...        ...      ...      ...      ...      ...  
205.0   1691.0      NaN      NaN      NaN      NaN  
206.0   1677.0      NaN      NaN      NaN      NaN  
207.0   1681.0      NaN      NaN      NaN      NaN  
208.0   1681.0      NaN      NaN      NaN      NaN  
209.0   1682.0      NaN      NaN      NaN      NaN  

[210 rows x 82 columns]

In [59]:
# save labels
all_axons_indices = [range(167,220), range(220,259), range(259,300)]
# os.listdir(dest_dir)

all_axons_data = []
for axons_index in all_axons_indices:
    data = pd.read_csv(f'{dest_dir}/labels_ID{axons_index[0]}-{axons_index[-1]}.csv', header=[0,1], index_col=0)
    all_axons_data.append(data)
    
    print(data)
all_axons_data = pd.concat(all_axons_data, axis=1)


all_axons_data.to_csv(f'{dest_dir}/labels_ID_all{all_axons_indices[0][0]}-{all_axons_indices[-1][-1]}.csv')

all_axons_data


      Axon_167          Axon_168          Axon_169          Axon_170           \
      anchor_x anchor_y anchor_x anchor_y anchor_x anchor_y anchor_x anchor_y   
0.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
2.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
...        ...      ...      ...      ...      ...      ...      ...      ...   
205.0      NaN      NaN   1046.0      7.0      NaN      NaN      NaN      NaN   
206.0      NaN      NaN   1049.0      7.0      NaN      NaN      NaN      NaN   
207.0      NaN      NaN   1051.0      7.0      NaN      NaN      NaN      NaN   
208.0      NaN      NaN   1016.0      4.0      NaN      NaN      NaN      NaN   
209.0      NaN      NaN   10

Axon_167          Axon_168          Axon_169          Axon_170           \
      anchor_x anchor_y anchor_x anchor_y anchor_x anchor_y anchor_x anchor_y   
0.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
2.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4.0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
...        ...      ...      ...      ...      ...      ...      ...      ...   
205.0      NaN      NaN   1046.0      7.0      NaN      NaN      NaN      NaN   
206.0      NaN      NaN   1049.0      7.0      NaN      NaN      NaN      NaN   
207.0      NaN      NaN   1051.0      7.0      NaN      NaN      NaN      NaN   
208.0      NaN      NaN   1016.0      4.0      NaN      NaN      NaN      NaN   
209.0      NaN      NaN   1031.0      2.0      NaN      NaN      NaN      NaN   

      Axon_171           ... Axon_295          Axon_296          Axon_297  \
      anchor_x anchor_y  ... anchor_x anchor_y anchor_x anchor_y anchor_x   
0.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
1.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
2.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
3.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
4.0        NaN      NaN  ...      NaN      NaN      NaN      NaN      NaN   
...        ...      ...  ...      ...      ...      ...      ...      ...   
205.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    546.0   
206.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    547.0   
207.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    545.0   
208.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    547.0   
209.0      NaN      NaN  ...      NaN      NaN      NaN      NaN    544.0   

               Axon_298          Axon_299           
      anchor_y anchor_x anchor_y anchor_x anchor_y  
0.0        NaN      NaN      NaN      NaN      NaN  
1.0        NaN      NaN      NaN      NaN      NaN  
2.0        NaN      NaN      NaN      NaN      NaN  
3.0        NaN      NaN      NaN      NaN      NaN  
4.0        NaN      NaN      NaN      NaN      NaN  
...        ...      ...      ...      ...      ...  
205.0   1691.0      NaN      NaN      NaN      NaN  
206.0   1677.0      NaN      NaN      NaN      NaN  
207.0   1681.0      NaN      NaN      NaN      NaN  
208.0   1681.0      NaN      NaN      NaN      NaN  
209.0   1682.0      NaN      NaN      NaN      NaN  

[210 rows x 266 columns]

In [28]:
for axon in all_axons_data.columns.unique(0):
    ax_data = all_axons_data[axon]
    print(ax_data.shape)
    print(all_axons_data.index.values[:, np.newaxis].shape)
    dat = np.concatenate([all_axons_data.index.values[:, np.newaxis], ax_data], axis=-1)
    viewer.add_points(name=axon, data=dat)


(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(101, 1)
(101, 2)
(

In [ ]:
# SAVE AXONS
print('SAVING...')
axons_index = range(100)

# backup for safety always created new
dir_name = f'{outp_path}/labelled_axons_backup/labelled_axons n{len(axons_index)}.{time.ctime(time.time())}'
os.makedirs(dir_name, exist_ok=True)

all_axons_df = []
for ax_i in axons_index:
    name = f'Axon_{ax_i:0>3}'
    # old labelling, save base object as .npy, Used for loading axon bboxes in this script
    labels = np.array(viewer.layers[name].data)
    np.save(f'{dir_name}/{name}.npy', labels)   # safety backup
    np.save(f'{outp_path}/labelled_axons/{name}.npy', labels)   # what's actually used

    # make dataframe to save all information
    timepoints = labels[:,0,0].astype(int)
    print(f'{name} n={viewer.layers[name].nshapes}...', end='')
    # create a pandas dataframe for this specific axons
    idx = pd.MultiIndex.from_product([(name,),
                                    ('anchor_x', 'anchor_y', 'extend', 
                                     'topleft_x', 'topleft_y', 
                                     'bottomright_x', 'bottomright_y', 
                                     'distance', 'rel_distance',
                                     'col_r', 'col_g', 'col_b',)])
    dat = pd.DataFrame(index=range(sizet), columns=idx)
    
    # populate the dataframe with all the relvent bbox information
    color = viewer.layers[name].edge_color
    dat.loc[timepoints, (name, ['col_r', 'col_g', 'col_b'])] = color[:,:-1]
        
    start = labels[:, :,1:].min(axis=1).astype(int)
    end = labels[:,:,1:].max(axis=1).astype(int)
    dat.loc[timepoints, (name, 'topleft_x')] = start[:, 1]
    dat.loc[timepoints, (name, 'topleft_y')] = start[:, 0]
    dat.loc[timepoints, (name, 'bottomright_x')] = end[:, 1]
    dat.loc[timepoints, (name, 'bottomright_y')] = end[:, 0]

    x_dist, y_dist = end[:,1]-start[:,1], end[:,0]-start[:,0]
    extend = ((x_dist+y_dist)//4).astype(int)
    dat.loc[timepoints, (name, 'extend')] = extend

    anchor_x = (start[:,1] + extend).astype(int)
    anchor_y = (start[:,0] + extend).astype(int)
    dat.loc[timepoints, (name, 'anchor_x')] = anchor_x
    dat.loc[timepoints, (name, 'anchor_y')] = anchor_y
    # this is the old way of computing distance: simply eucl. 
    dist = np.sqrt((anchor_x-target[1])**2 + (anchor_y-target[0])**2)
    dat.loc[timepoints, (name, 'distance')] = dist
    dat.loc[timepoints, (name, 'rel_distance')] = dist[np.where(timepoints == min(timepoints))] - dist

    all_axons_df.append(dat)
    print('Ok.')
all_axons_df = pd.concat(all_axons_df, axis=1).sort_index(axis=1)
all_axons_df.to_csv(f'{outp_path}/labelled_axons_eucldists_new.csv')


In [ ]:
from skimage.color import gray2rgb
from skimage import img_as_float
from skimage.draw import rectangle_perimeter
from skimage import img_as_uint

redchannel_float = img_as_float(redchannel)

In [ ]:
rgb_image = np.zeros((sizet, sizey, sizex, 3), float)
rgb_image[:,:,:,0] = redchannel_float
rgb_image /= rgb_image.max()

In [ ]:
# LOAD AXONS AS LAYERS, or rgb image, show which timepoint is lablled 
as_shapes = True
print('LOADING axon bboxes....')
load_axons_range = range(100)

labelled = np.zeros((load_axons_range[-1]+1,sizet))
for i in load_axons_range:
    name = f'Axon_{i:0>3}'
    print(f'{name}...', end='')
    col = np.random.rand(1,3)
    dat = np.load(f'{outp_path}/labelled_axons/{name}.npy', )
    labelled[i,:] = [1 if t in dat[:,:,0].flatten() else 0 for t in range(sizet)]   # for plotting
    # dat[:,:,1] -= ymin
    # dat[:,:,2] -= xmin
    
    if not dat.size:
        print('!!!Empty!!!\n')
        continue
    dupl_label = any([(dat[:,:,0]==t).sum()>4 for t in range(sizet)])
    if dupl_label:
        [print(f't: {t} - More than one label!') for t in range(sizet) if (dat[:,:,0]==t).sum()>4]        
        print()
    else:
        print('Ok')
    if as_shapes:
        viewer.add_shapes(dat, name=name, shape_type='rectangle', edge_color=col, face_color=np.array([1,1,1,.9]), edge_width=5, visible=True, opacity=.9)
    else:
        start = dat[:, :,1:].min(axis=1).astype(int)
        end = dat[:,:,1:].max(axis=1).astype(int)
        x_start = start[:, 1]
        y_start = start[:, 0]
        x_end = end[:, 1]
        y_end = end[:, 0]
        
#         col = 1 - .35*(1-col) 
        axon_timepoints = dat[:,:,0].astype(int)
        for i, t in enumerate(range(axon_timepoints.min(), axon_timepoints.max()+1)):
            for w in range(2):
                rr, cc = rectangle_perimeter((y_start[i]+w, x_start[i]+w), end=(y_end[i]+w, x_end[i]+w), 
                                             shape=rgb_image.shape[1:])
                rgb_image[t-1, rr, cc] = col

plt.figure(figsize=(13,7), facecolor='k')
plt.imshow(labelled.T, cmap=plt.get_cmap('cividis'))
ax = plt.gca()

ax.set_yticks(np.arange(sizet))
ax.set_yticklabels(['' if t%5 else t for t in range(sizet)], color='white')
ax.set_ylabel('timepoint', color='white', fontsize=14)

ax.set_xticks(np.arange(load_axons_range[-1]+1))
ax.set_xticklabels(['' if t%2 else t for t in range(load_axons_range[-1]+1)], color='white')
ax.set_xlabel('Axon ID', color='white', fontsize=14)

ax.set_title('Yellow means there is an axon label for this timepoint', color='white', fontsize=18)
ax.set_yticks(np.arange(sizet))
ax.set_yticklabels(['' if t%4 else t+1 for t in range(sizet)], color='white')
ax.set_ylabel('timepoint', color='white', fontsize=14)

In [ ]:
viewer.add_image(rgb_image)